# Phase 1: Baseline Evaluation with SimplePlanner
**Goal:** Establish baseline performance metrics using SimplePlanner on nuPlan scenarios

**Team:** Karina Shah, Dhruvina Gujarati, Nilay Kumar, Nishanth Krishna Churchmal

**Course:** CSE 475 - Fall 2025


In [1]:

# Setup
import sys
sys.path.append('/Users/dhruvinagujarati/nuplan-devkit')

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

# Verify environment variables
print("Environment Check:")
print(f"NUPLAN_DATA_ROOT: {os.getenv('NUPLAN_DATA_ROOT')}")
print(f"NUPLAN_MAPS_ROOT: {os.getenv('NUPLAN_MAPS_ROOT')}")
print(f"NUPLAN_EXP_ROOT: {os.getenv('NUPLAN_EXP_ROOT')}")

Environment Check:
NUPLAN_DATA_ROOT: /home/kshah26_/nuplan/dataset
NUPLAN_MAPS_ROOT: /home/kshah26_/nuplan/dataset/maps
NUPLAN_EXP_ROOT: /home/kshah26_/nuplan/exp


In [2]:
# Import nuPlan components
from nuplan.planning.simulation.planner.simple_planner import SimplePlanner
from nuplan.planning.scenario_builder.nuplan_db.nuplan_scenario_builder import NuPlanScenarioBuilder
from nuplan.planning.scenario_builder.nuplan_db.nuplan_scenario_utils import ScenarioMapping
from nuplan.common.actor_state.ego_state import EgoState
from nuplan.planning.simulation.trajectory.trajectory_sampling import TrajectorySampling

print("nuPlan imports successful")

# Set up paths
NUPLAN_DATA_ROOT = os.getenv('NUPLAN_DATA_ROOT')
NUPLAN_MAPS_ROOT = os.getenv('NUPLAN_MAPS_ROOT')
NUPLAN_DB_FILES = os.path.join(NUPLAN_DATA_ROOT, 'nuplan-v1.1/splits/mini')
NUPLAN_MAP_VERSION = 'nuplan-maps-v1.0'

print(f"Database path: {NUPLAN_DB_FILES}")
print(f"Maps path: {NUPLAN_MAPS_ROOT}")

# Check if database files exist
db_files = list(Path(NUPLAN_DB_FILES).glob('*.db'))
print(f"Found {len(db_files)} database files")
print(f"  Example: {db_files[0].name if db_files else 'None'}")

nuPlan imports successful
Database path: /home/kshah26_/nuplan/dataset/nuplan-v1.1/splits/mini
Maps path: /home/kshah26_/nuplan/dataset/maps
Found 64 database files
  Example: 2021.06.03.12.02.06_veh-35_00233_00609.db


In [3]:
# Initialize SimplePlanner
print("Initializing SimplePlanner...")
simple_planner = SimplePlanner(
    horizon_seconds=10.0,
    sampling_time=0.5,
    acceleration=[0.0, 0.0],
    max_velocity=5.0,
    steering_angle=0.0
)
print("✓ SimplePlanner initialized")

# Let's use a direct database approach with SQLAlchemy
print("\nConnecting to database...")
import sqlite3

# Connect to first database file
db_file = db_files[0]
conn = sqlite3.connect(str(db_file))
cursor = conn.cursor()

print(f"✓ Connected to: {db_file.name}")

# Query basic info
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
print(f"✓ Found {len(tables)} tables in database")

# Get scenario information
cursor.execute("SELECT COUNT(*) FROM scenario_tag;")
num_scenarios = cursor.fetchone()[0]
print(f"✓ Found {num_scenarios} tagged scenarios")

# Get scenario types
cursor.execute("SELECT DISTINCT type FROM scenario_tag LIMIT 20;")
scenario_types = cursor.fetchall()
print(f"\nSample scenario types ({len(scenario_types)} shown):")
for i, (stype,) in enumerate(scenario_types, 1):
    print(f"  {i}. {stype}")

# Get a sample scenario
cursor.execute("""
    SELECT token, type, lidar_pc_token 
    FROM scenario_tag 
    LIMIT 5
""")
sample_scenarios = cursor.fetchall()
print(f"\n✓ Sample scenarios loaded: {len(sample_scenarios)}")

conn.close()

Initializing SimplePlanner...
✓ SimplePlanner initialized

Connecting to database...
✓ Connected to: 2021.06.03.12.02.06_veh-35_00233_00609.db
✓ Found 12 tables in database
✓ Found 9866 tagged scenarios

Sample scenario types (20 shown):
  1. low_magnitude_speed
  2. medium_magnitude_speed
  3. high_magnitude_speed
  4. near_high_speed_vehicle
  5. near_long_vehicle
  6. near_construction_zone_sign
  7. near_trafficcone_on_driveable
  8. near_pedestrian_on_crosswalk
  9. on_stopline_traffic_light
  10. on_intersection
  11. on_traffic_light_intersection
  12. on_pickup_dropoff
  13. starting_straight_traffic_light_intersection_traversal
  14. starting_straight_stop_sign_intersection_traversal
  15. traversing_pickup_dropoff
  16. traversing_intersection
  17. traversing_crosswalk
  18. traversing_traffic_light_intersection
  19. following_lane_with_slow_lead
  20. accelerating_at_traffic_light_with_lead

✓ Sample scenarios loaded: 5
